# Laboratorio 3 - Análisis de Asociación

Integrantes:
- Derek Suarez - B97775
- Emmanuel Zúñiga - B98729

## Tutorial

### Carga de Librerias 📚

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Carga del Dataset 📅

In [2]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')

In [3]:
# Se verifican los primeros 10 registros del dataset
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


### Preprocesamiento de Datos 🧹

A continuación se realiza una limpieza del dataset, entre algunas de ellas se puede destacar: eliminación de espacios en la columna "Description", y eliminación de filas sin número de factura y las transacciones de crédito.

In [4]:
df['Description'] = df['Description'].str.strip() # Eliminación de espacios
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

Después de la limpieza, se necesita consolidar los artículos en 1 transacción por fila con cada producto 1 codificado en caliente. Con el objetivo de reducir el dataset.

In [5]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

Se verifican los primeros 10 registros del dataset generado.

In [7]:
basket.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537468,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Se aplica transformación de los datos para convertir cualquier valor mayor a 1, y 0 en 1 y 0 respectivamente.

In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

Se verifican los primeros 10 registros del dataset

In [9]:
basket_sets.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537468,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Algoritmo Apriori 🔢


Con los datos ya debidamente estructurados, se generan itemsets para aquellos con una frecuencia mayor al 6%

In [14]:
frequent_itemsets = apriori(basket_sets, min_support=0.06, use_colnames=True)

El último paso es generar las reglas con su correspondiente apoyo, confianza y elevación:

In [15]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

Se verifican los primeros 10 registros del dataset.

In [16]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
6,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
7,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
8,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273
9,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.099490,0.071429,0.571429,5.743590,0.058992,2.101190


A continuación se realiza un análisis de los datos obtenidos, para ello, se evaluan varias condiciones como lift, support, confidence, entre otros. Esto nos permite intuir aspectos del conjunto de datos.

Se filtran los registros que tienen un lift mayor o igual a 6, y un valor para cofidence mayor o igual al 80%

In [17]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
6,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
7,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
31,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
32,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
33,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
34,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
35,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE PINK),0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456
36,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194


Nótese que a partir del filtrado de los datos se puede observar como ciertos items suelen venir en conjunto, por ejemplo, las alarmas verde y rojas suelen comprarse juntas, similarmente con los demás items, donde podemos observar los conjuntos de aquellas transacciones que suelen estar relacionadas.

In [18]:
print(f"Frecuencia de alarmas verdes: {basket['ALARM CLOCK BAKELIKE GREEN'].sum()}")

print(f"Frecuencia de alarmas rojas: {basket['ALARM CLOCK BAKELIKE RED'].sum()}")


Frecuencia de alarmas verdes: 340.0
Frecuencia de alarmas rojas: 316.0


Igualmente, se pueden filtrar los resultados con respecto a la columna 'Country', lo cual, permite observar las combinaciones de items más populares por país. En este caso, se filtraran los resultados para Francia.

In [22]:
basket2 = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]    

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
8,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
9,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
10,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273
11,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.099490,0.071429,0.571429,5.743590,0.058992,2.101190


¿Qué nos dice esta tabla?
A partir de los resultados presentados anteriormente, se puede verificar como en Francia existe una preferencia por las alarmas, independientemente de su color. Lo usual, es que en Francia se suele comprar en conjunto alarmas verdes y alarmas rosas.

## MLextend Apriori

A continuación, se presentan los ejemplos de uso para el algoritmo apriori según la libreria MLextend.

### Carga de Librerias 📚

In [27]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

### Carga del Dataset 📅

A continuación se genera un dataset con nombres de videojuegos.

In [25]:
dataset = [['God of War: Ragnarök', 'Elden Ring', 'Red Dead Redemption: II', 'Forza: Horizon', 'Blasphemous', 'Cyberpunk 2077'],
          ['Hogwarts Legacy', 'Elden Ring', 'Red Dead Redemption: II', 'Forza: Horizon', 'Blasphemous', 'Cyberpunk 2077'],
          ['God of War: Ragnarök', 'Final Fantasy XVI', 'Forza: Horizon', 'Blasphemous'],
          ['God of War: Ragnarök', 'Diablo IV', 'Ark 2', 'Forza: Horizon', 'Cyberpunk 2077'],
          ['Ark 2', 'Elden Ring', 'Elden Ring', 'Forza: Horizon', 'Minecraft', 'Blasphemous']]

A continuación, se verifican las apariciones de los videojuegos para cada "transacción", puede determinarse la aparición de cada videojuego en las transacciones según su valor, ya sea (True o False).

In [26]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Ark 2,Blasphemous,Cyberpunk 2077,Diablo IV,Elden Ring,Final Fantasy XVI,Forza: Horizon,God of War: Ragnarök,Hogwarts Legacy,Ice cream,Red Dead Redemption: II
0,False,True,True,False,True,False,True,True,False,False,True
1,False,True,True,False,True,False,True,False,True,False,True
2,False,True,False,False,False,True,True,True,False,False,False
3,True,False,True,True,False,False,True,True,False,False,False
4,True,True,False,False,True,False,True,False,False,True,False


Posteriormente, verificamos aquellos items com un soporte de al menos 60%

In [29]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Blasphemous)
1,0.6,(Cyberpunk 2077)
2,0.6,(Elden Ring)
3,1.0,(Forza: Horizon)
4,0.6,(God of War: Ragnarök)
5,0.6,"(Blasphemous, Elden Ring)"
6,0.8,"(Forza: Horizon, Blasphemous)"
7,0.6,"(Forza: Horizon, Cyberpunk 2077)"
8,0.6,"(Forza: Horizon, Elden Ring)"
9,0.6,"(Forza: Horizon, God of War: Ragnarök)"


A partir de los datos mostrados anteriormente, se puede observar como Blasphemous y Elden Ring suelen ser comprados en conjuntos, similarmente a Forza: Horizon y Blasphemous, entre otros.

En caso de que se quiere ahorrar espacio, resulta útil representar los datos transaccionales en formato sparse.

In [30]:
oht_ary = te.fit(dataset).transform(dataset, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df

,Ark 2,Blasphemous,Cyberpunk 2077,Diablo IV,Elden Ring,Final Fantasy XVI,Forza: Horizon,God of War: Ragnarök,Hogwarts Legacy,Ice cream,Red Dead Redemption: II
0,0,1,1,0,1,0,True,1,0,0,1
1,0,1,1,0,1,0,True,0,1,0,1
2,0,1,0,0,0,1,True,1,0,0,0
3,1,0,1,1,0,0,True,1,0,0,0
4,1,1,0,0,1,0,True,0,0,1,0


In [32]:
apriori(sparse_df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Blasphemous)
1,0.6,(Cyberpunk 2077)
2,0.6,(Elden Ring)
3,1.0,(Forza: Horizon)
4,0.6,(God of War: Ragnarök)
5,0.6,"(Blasphemous, Elden Ring)"
6,0.8,"(Forza: Horizon, Blasphemous)"
7,0.6,"(Forza: Horizon, Cyberpunk 2077)"
8,0.6,"(Forza: Horizon, Elden Ring)"
9,0.6,"(Forza: Horizon, God of War: Ragnarök)"


Similarmente a la tabla presentada más arriba, aqui se muestran aquellos itemsets con un soporte de al menos el 60% en el dataset transaccional.

La implementación de estás metodologías, permiten el estudio exhaustivo de datasets transaccionales, esto permite entre otros casos: aplicación de promociones, distribución de los items en la tiendra, entre otros.